In [85]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import pandas as pd
from es_db import get_rtc_urls, get_rtc_docs, get_search_client, get_static_rtc_docs
from tqdm import tqdm
import concurrent.futures

In [7]:
df_data = pd.read_csv('../OPERA-RTC_CalVal_S1_Scene_IDs.csv')
df_data['S1_Scene_IDs'] = df_data.S1_Scene_IDs.map(lambda s: s.split())
df_data = df_data.explode('S1_Scene_IDs').reset_index(drop=True)
df_data.head()

,Site,Orbit_Direction,Orbital_Path,CalVal_Module,S1_Scene_IDs
0,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20200324T015024_20200324T0150...
1,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20201107T015034_20201107T0151...
2,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20220501T015035_20220501T0151...
3,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20200604T015028_20200604T0150...
4,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20210130T015031_20210130T0150...


In [8]:
sample_slc_id = df_data.S1_Scene_IDs[1]
sample_slc_id

'S1A_IW_SLC__1SDV_20201107T015034_20201107T015102_035136_0419CE_F4EF,'

In [9]:
slc_ids = df_data.S1_Scene_IDs.tolist()

In [10]:
rtc_docs = get_rtc_docs(slc_ids[0])

In [11]:
len(rtc_docs)

28

In [30]:
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    rtc_docs_lst = list(tqdm(executor.map(get_rtc_docs, slc_ids[:]), total=len(slc_ids)))

In [45]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    static_rtc_docs_lst = list(tqdm(executor.map(get_static_rtc_docs, rtc_docs_lst[:]), total=len(rtc_docs_lst)))

 41%|████▉       | 661/1623 [02:46<03:15,  4.91it/s]

T034-071081-IW1 does not have a entry in ES
T034-071081-IW2 does not have a entry in ES


 41%|████▉       | 663/1623 [02:46<03:05,  5.18it/s]

T034-071080-IW1 does not have a entry in ES
T034-071081-IW3 does not have a entry in ES
T034-071080-IW3 does not have a entry in ES
T034-071080-IW2 does not have a entry in ES


 42%|█████       | 689/1623 [02:53<03:45,  4.15it/s]

...

 71%|███████▊   | 1157/1623 [04:41<01:25,  5.48it/s]

T107-227900-IW2 does not have a entry in ES
T107-227898-IW2 does not have a entry in ES
T107-227895-IW2 does not have a entry in ES
T107-227895-IW1 does not have a entry in ES
T107-227897-IW3 does not have a entry in ES
T107-227896-IW2 does not have a entry in ES
T107-227898-IW1 does not have a entry in ES
T107-227895-IW3 does not have a entry in ES
T107-227901-IW1 does not have a entry in ES
T107-227900-IW1 does not have a entry in ES
T107-227896-IW3 does not have a entry in ES
T107-227897-IW3 does not have a entry in ES
T107-227898-IW3 does not have a entry in ES
T107-227899-IW3 does not have a entry in ES
T107-227899-IW1 does not have a entry in ES
T107-227895-IW2 does not have a entry in ES
T107-227900-IW3 does not have a entry in ES
T107-227898-IW2 does not have a entry in ES
T107-227899-IW2 does not have a entry in ES
T107-227897-IW2 does not have a entry in ES
T107-227896-IW1 does not have a entry in ES
T107-227897-IW1 does not have a entry in ES
T107-227896-IW2 does not have a 

 72%|███████▉   | 1163/1623 [04:42<01:07,  6.86it/s]

T107-227895-IW1 does not have a entry in ES


100%|███████████| 1623/1623 [06:29<00:00,  4.16it/s]


In [62]:
def get_rtc_urls(rtc_docs_lst: list[dict]) -> dict:
    urls = [{rtc_doc['id']: rtc_doc['metadata']['product_urls']
             for rtc_doc in rtc_docs} for rtc_docs in rtc_docs_lst]
    return urls

In [87]:
out_data = get_rtc_urls(rtc_docs_lst)

In [88]:
opera_rtc_ids = [' '.join(list(d.keys())) for d in out_data]
n_prods = [len(list(d.keys())) for d in out_data]
urls_split = {}
for k in range(5):
    urls_split[k] = [' '.join([val[k] for _, val in d.items()]) for d in out_data]

In [89]:
df_data['opera_rtc_ids'] = opera_rtc_ids
df_data['h5_url'] = urls_split[0]
df_data['browse_url'] = urls_split[1]
df_data['vh_url'] = urls_split[2]
df_data['vv_url'] = urls_split[3]
df_data['mask_url'] = urls_split[4]
df_data['n_bursts'] = n_prods

In [90]:
df_data.head()

,Site,Orbit_Direction,Orbital_Path,CalVal_Module,S1_Scene_IDs,opera_rtc_ids,h5_url,browse_url,vh_url,vv_url,mask_url,n_bursts
0,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20200324T015024_20200324T0150...,OPERA_L2_RTC-S1_T064-135519-IW2_20200324T01502...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,28
1,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20201107T015034_20201107T0151...,OPERA_L2_RTC-S1_T064-135527-IW2_20201107T01505...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,28
2,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20220501T015035_20220501T0151...,OPERA_L2_RTC-S1_T064-135520-IW1_20220501T01504...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,28
3,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20200604T015028_20200604T0150...,OPERA_L2_RTC-S1_T064-135523-IW1_20200604T01503...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,28
4,California,Ascending,64,Absolute Geolocation Evaluation,S1A_IW_SLC__1SDV_20210130T015031_20210130T0150...,OPERA_L2_RTC-S1_T064-135523-IW2_20210130T01504...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,28


In [96]:
df_data.to_csv('opera_rtc_table.csv.zip')

In [97]:
df_data.n_bursts.sum()

43687

In [98]:
len(static_rtc_docs_lst)

1623

In [99]:
def get_url(data_dict):
    if data_dict:
        return ' '.join(data_dict['metadata']['product_urls'])
    return ''
burst_id_keyed = {burst_id: val for data_lst in static_rtc_docs_lst for data_dict in data_lst for (burst_id, val) in data_dict.items()}
static_data = [{'burst_id': burst_id, 'urls': get_url(val)} for (burst_id, val) in burst_id_keyed.items()]
df_static = pd.DataFrame(static_data)
df_static.head()

,burst_id,urls
0,T064-135525-IW1,https://opera-pst-rs-pop1.s3.us-west-2.amazona...
1,T064-135527-IW1,https://opera-pst-rs-pop1.s3.us-west-2.amazona...
2,T064-135520-IW1,https://opera-pst-rs-pop1.s3.us-west-2.amazona...
3,T064-135524-IW1,https://opera-pst-rs-pop1.s3.us-west-2.amazona...
4,T064-135524-IW3,https://opera-pst-rs-pop1.s3.us-west-2.amazona...


In [101]:
df_static.shape

(353, 2)

In [100]:
df_static.to_csv('opera_rtc_static_table.csv.zip')